## Inference Pepline:


In [2]:

## 1-tools:
# tools imports

import numpy as np
import pandas as pd
import polars as pl

import torch
from torch import nn
from torch.nn import functional as F

import os
import sys
from pathlib import Path

import sys

try:
    import kaggle_evaluation.rsna_inference_server
except:
    sys.path.append('/kaggle/input/rsna-intracranial-aneurysm-detection')
    import kaggle_evaluation.rsna_inference_server

import gc  # garbage collection
from typing import List, Tuple, Optional, Dict  # hints

print('imports ok')

ModuleNotFoundError: No module named 'kaggle_evaluation'

## 1 Pydicom .dcm files to .nzp files¶

In [4]:
import os, glob
import numpy as np
import pydicom
from scipy.ndimage import zoom

# -------------------------
# Config
# -------------------------
TARGET_SHAPE = (32, 256, 256)  # (D,H,W)
WINDOWS = [(40, 80), (600, 2800)]  # WL, WW
BBOX_MARGIN = 10

# -------------------------
# Helpers
# -------------------------
def dcm_key_sorter(path):
    """Sort slices by InstanceNumber or Z position."""
    try:
        ds = pydicom.dcmread(path, stop_before_pixels=True)
        if "InstanceNumber" in ds:
            return int(ds.InstanceNumber)
        if "ImagePositionPatient" in ds:
            return float(ds.ImagePositionPatient[2])
    except:
        pass
    return path

def load_dicom_volume(series_dir):
    """Load all slices of a DICOM series as a 3D numpy array."""
    files = sorted(glob.glob(os.path.join(series_dir, "*.dcm")), key=dcm_key_sorter)
    slices = []
    for fp in files:
        try:
            ds = pydicom.dcmread(fp, force=True)
            arr = ds.pixel_array.astype(np.float32)
            if hasattr(ds, "RescaleSlope") and hasattr(ds, "RescaleIntercept"):
                arr = arr * float(ds.RescaleSlope) + float(ds.RescaleIntercept)
            if arr.ndim == 3:
                arr = arr[..., 0]
            slices.append(arr)
        except:
            continue
    if not slices:
        raise ValueError(f"No valid DICOMs found in {series_dir}")
    return np.stack(slices, axis=0).astype(np.float32)  # (D,H,W)

def apply_window(vol, WL, WW):
    """Apply intensity window."""
    lower, upper = WL - WW/2.0, WL + WW/2.0
    clipped = np.clip(vol, lower, upper)
    return ((clipped - lower) / max(1e-12, (upper - lower))).astype(np.float32)

def center_crop(vol, target_shape):
    """Crop or pad to match target shape at center."""
    d,h,w = vol.shape
    td,th,tw = target_shape
    out = np.zeros(target_shape, dtype=vol.dtype)

    d0 = max((d-td)//2,0); d1 = d0+td
    h0 = max((h-th)//2,0); h1 = h0+th
    w0 = max((w-tw)//2,0); w1 = w0+tw

    ds = max((td-d)//2,0); de = ds+d
    hs = max((th-h)//2,0); he = hs+h
    ws = max((tw-w)//2,0); we = ws+w

    out[ds:de, hs:he, ws:we] = vol[d0:d1, h0:h1, w0:w1]
    return out

def resize_volume(vol, target_shape):
    """Resize to target shape using scipy zoom."""
    factors = [t/c for t,c in zip(target_shape, vol.shape)]
    return zoom(vol, factors, order=1).astype(np.float32)

# -------------------------
# Main processor
# -------------------------
def process_series_to_npz(series_dir, target_shape=TARGET_SHAPE, windows=WINDOWS):
    """
    Load dicom series -> apply windowing -> resize -> return npz-like dict.
    Returns dict with 'image' key for inference.
    """
    vol = load_dicom_volume(series_dir)  # (D,H,W)

    # Windowed channels
    chans = [apply_window(vol, wl, ww) for wl, ww in windows]  # list of (D,H,W)

    # Stack channels
    vol_cdhw = np.stack(chans, axis=0)  # (C,D,H,W)

    # Resize each channel to target
    C = vol_cdhw.shape[0]
    out = np.zeros((C, *target_shape), dtype=np.float32)
    for c in range(C):
        out[c] = resize_volume(vol_cdhw[c], target_shape)
    #print(f'/////////////////////volume shape ={out.shape}////////////////////////')
    out = out[:1] #<============ this line is added
    return  out#{"image": out}

cuda is availabe that is True


In [8]:
##  2-ReUse The training setting:

##  2-ReUse The training setting:

In [ ]:
import torch
import numpy as np
import pydicom, glob, os
from scipy.ndimage import zoom

# ----------------------------
# Reuse your training settings
# ----------------------------
TARGET_SHAPE = (32, 256, 256)
WINDOWS = [(40, 80), (600, 2800)]
DEVICE = "cuda"

def apply_window(vol, WL, WW):
    lower = WL - WW/2.0
    upper = WL + WW/2.0
    clipped = np.clip(vol, lower, upper)
    scaled = (clipped - lower) / max(1e-12, (upper - lower))
    return scaled.astype(np.float32)

def resample_volume(vol, target_shape):
    dz = target_shape[0] / vol.shape[0]
    dh = target_shape[1] / vol.shape[1]
    dw = target_shape[2] / vol.shape[2]
    return zoom(vol, (dz, dh, dw), order=1)

def load_dicom_series(series_path):
    files = sorted(glob.glob(os.path.join(series_path, "*.dcm")))
    slices = []
    for f in files:
        ds = pydicom.dcmread(f)
        arr = ds.pixel_array.astype(np.float32)
        if hasattr(ds, "RescaleSlope") and hasattr(ds, "RescaleIntercept"):
            arr = arr * float(ds.RescaleSlope) + float(ds.RescaleIntercept)
        slices.append(arr)
    vol = np.stack(slices, axis=0)
    return vol

def preprocess_series(series_path):
    vol = load_dicom_series(series_path)

    # multi-window channels
    chans = []
    for wl, ww in WINDOWS:
        chans.append(apply_window(vol, wl, ww))
    vol_cdhw = np.stack(chans, axis=0)  # (C,D,H,W)

    # resample each channel
    out = np.zeros((vol_cdhw.shape[0], *TARGET_SHAPE), dtype=np.float32)
    for c in range(vol_cdhw.shape[0]):
        out[c] = resample_volume(vol_cdhw[c], TARGET_SHAPE)

    # scale back to [0,1]
    out = np.clip(out, 0, 1)

    return out

def run_inference(model, series_path, class_names):
    arr = preprocess_series(series_path)
    x = torch.tensor(arr, dtype=torch.float32).unsqueeze(0).to(DEVICE)  # (1,C,D,H,W)

    with torch.no_grad():
        logits = model(x)
        probs = torch.sigmoid(logits).cpu().numpy()[0]

    return dict(zip(class_names, probs))



## 3-Model Definition:


In [ ]:
# this models is borrow from https://github.com/Tencent/MedicalNet/blob/master/models/resnet.py
import torch
import torch.nn as nn
import torch.nn.functional as F
from functools import partial

__all__ = [
    'resnet10', 'resnet18', 'resnet34', 'resnet50',
    'resnet101', 'resnet152', 'resnet200'
]

def conv3x3x3(in_planes, out_planes, stride=1, dilation=1):
    """3x3x3 convolution with padding"""
    return nn.Conv3d(
        in_planes,
        out_planes,
        kernel_size=3,
        dilation=dilation,
        stride=stride,
        padding=dilation,
        bias=False
    )


def downsample_basic_block(x, planes, stride):
    """Downsample using avg pooling + zero padding"""
    out = F.avg_pool3d(x, kernel_size=1, stride=stride)
    zero_pads = torch.zeros(
        out.size(0), planes - out.size(1), out.size(2), out.size(3), out.size(4),
        device=x.device, dtype=x.dtype
    )
    out = torch.cat([out, zero_pads], dim=1)
    return out


class BasicBlock(nn.Module):
    expansion = 1
    def __init__(self, inplanes, planes, stride=1, dilation=1, downsample=None):
        super().__init__()
        self.conv1 = conv3x3x3(inplanes, planes, stride, dilation)
        self.bn1 = nn.BatchNorm3d(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3x3(planes, planes, dilation=dilation)
        self.bn2 = nn.BatchNorm3d(planes)
        self.downsample = downsample

    def forward(self, x):
        residual = x
        out = self.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        if self.downsample is not None:
            residual = self.downsample(x)
        out = self.relu(out + residual)
        return out


class Bottleneck(nn.Module):
    expansion = 4
    def __init__(self, inplanes, planes, stride=1, dilation=1, downsample=None):
        super().__init__()
        self.conv1 = nn.Conv3d(inplanes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm3d(planes)

        self.conv2 = nn.Conv3d(
            planes, planes, kernel_size=3, stride=stride, padding=dilation,
            dilation=dilation, bias=False
        )
        self.bn2 = nn.BatchNorm3d(planes)

        self.conv3 = nn.Conv3d(planes, planes * 4, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm3d(planes * 4)

        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample

    def forward(self, x):
        residual = x
        out = self.relu(self.bn1(self.conv1(x)))
        out = self.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        if self.downsample is not None:
            residual = self.downsample(x)
        out = self.relu(out + residual)
        return out


class ResNet3D(nn.Module):
    def __init__(self, block, layers, num_classes=1, in_channels=1, segmentation= not  False):
        super().__init__()
        self.inplanes = 64
        self.segmentation = segmentation

        self.conv1 = nn.Conv3d(in_channels, 64, kernel_size=7,
                               stride=(2, 2, 2), padding=(3, 3, 3), bias=False)
        self.bn1 = nn.BatchNorm3d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool3d(kernel_size=3, stride=2, padding=1)

        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2)
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2)

        self.avgpool = nn.AdaptiveAvgPool3d((1, 1, 1))
        self.fc = nn.Linear(512 * block.expansion, num_classes)

        if self.segmentation:
            #self.seg_head = nn.Conv3d(512 * block.expansion, num_classes, kernel_size=1)
            self.seg_head = nn.Sequential(  # this for resnet18
                nn.ConvTranspose3d(512, 256, 2, stride=2),
                nn.ReLU(inplace=True),
                nn.ConvTranspose3d(256, 128, 2, stride=2),
                nn.ReLU(inplace=True),
                nn.ConvTranspose3d(128, 64, 2, stride=2),
                nn.ReLU(inplace=True),
                nn.Conv3d(64, num_classes, 1)
            )
            self.seg_head2 = nn.Sequential(  # this for resnet50
                nn.ConvTranspose3d(
                    2048, 32, kernel_size=2, stride=2
                ),
                nn.BatchNorm3d(32),
                nn.ReLU(inplace=True),
                nn.Conv3d(32, 32, kernel_size=3, padding=1, bias=False),
                nn.BatchNorm3d(32),
                nn.ReLU(inplace=True),
                nn.Conv3d(32, 14, kernel_size=1, bias=False)
            )

    def _make_layer(self, block, planes, blocks, stride=1, dilation=1):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                nn.Conv3d(self.inplanes, planes * block.expansion,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm3d(planes * block.expansion)
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, dilation, downsample))
        self.inplanes = planes * block.expansion
        for _ in range(1, blocks):
            layers.append(block(self.inplanes, planes, dilation=dilation))
        return nn.Sequential(*layers)

    def forward(self, x):
        x=x.float()
        x_orig = x  # save original input
        x = self.relu(self.bn1(self.conv1(x)))
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        # classification
        cls_out = self.avgpool(x)
        cls_out = torch.flatten(cls_out, 1)
        cls_out = self.fc(cls_out)

        # segmentation
        seg_out = None
        if self.segmentation:
            seg_out = self.seg_head(x)
            # interpolate to match ORIGINAL input size
            seg_out = F.interpolate(
                seg_out,
                size=x_orig.shape[2:],  # <-- key: use original input shape
                mode='trilinear',
                align_corners=False
            )
            return cls_out.float(), seg_out.float()
        else:
            return cls_out.float()


# Factory functions
def resnet10(**kwargs):
    return ResNet3D(BasicBlock, [1,1,1,1], **kwargs)

def resnet18(**kwargs):
    return ResNet3D(BasicBlock, [2,2,2,2], **kwargs)

def resnet34(**kwargs):
    return ResNet3D(BasicBlock, [3,4,6,3], **kwargs)

def resnet50(**kwargs):
    return ResNet3D(Bottleneck, [3,4,6,3], **kwargs)

def resnet101(**kwargs):
    return ResNet3D(Bottleneck, [3,4,23,3], **kwargs)

def resnet152(**kwargs):
    return ResNet3D(Bottleneck, [3,8,36,3], **kwargs)

def resnet200(**kwargs):
    return ResNet3D(Bottleneck, [3,24,36,3], **kwargs)

## 4 Laoding Pretarnined models( .pt checkpoints):

In [ ]:
def load_series_as_array(series_path):
    """Load hidden test DICOM series → (D,H,W) float32."""
    dcm_files = sorted(glob.glob(f"{series_path}/*.dcm"))
    slices = []
    for f in dcm_files:
        dcm = pydicom.dcmread(f)
        img = dcm.pixel_array.astype(np.float32)
        slices.append(img)

    if not slices:
        return None

    volume = np.stack(slices, axis=0)
    volume = np.resize(volume, TARGET_SHAPE).astype(np.float32)

    # Suppose 'x' is your 4D array: [D, H, W]
    #x = torch.tensor(x, dtype=torch.float32)         # [D,H,W]
    #x = x.unsqueeze(0).unsqueeze(0)                 # [1,1,D,H,W] (batch + channel)
    #x = x.to(DEVICE)
    return volume

import os
import random
import numpy as np
import torch

def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # if using multi-GPU
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    os.environ["PYTHONHASHSEED"] = str(seed)

set_seed(42)
ckpt_path ='/kaggle/input/cv-all58/best_model.pt'
device ='cuda'
num_classes = 14
def _load_model_for_fold(model, ckpt_path, device, num_classes):
    model = model(num_classes=num_classes).to(device)
    model.eval()
    ckpt = torch.load(ckpt_path,
                      map_location=device,
                      weights_only=True)

    state_dict = ckpt.get("model_state_dict", ckpt)
    if any(k.startswith("module.") for k in state_dict.keys()):
        state_dict = {k.replace("module.", "", 1): v for k, v in state_dict.items()}

    missing, unexpected = model.load_state_dict(state_dict, strict=False)
    if missing or unexpected:
        print(f"[WARN] load_state_dict(strict=False) reported:")
        print(f"Missing keys ({len(missing)}): {missing[:10]}{' ...' if len(missing) > 10 else ''}")
        print(f"Unexpected keys ({len(unexpected)}): {unexpected[:10]}{' ...' if len(unexpected) > 10 else ''}")

    return model


## 5-Making The Prediction:

In [ ]:
# ==============
# Inference API
# ==============
import polars as pl
LABEL_COLS = [
    "Left Infraclinoid Internal Carotid Artery",
    "Right Infraclinoid Internal Carotid Artery",
    "Left Supraclinoid Internal Carotid Artery",
    "Right Supraclinoid Internal Carotid Artery",
    "Left Middle Cerebral Artery",
    "Right Middle Cerebral Artery",
    "Anterior Communicating Artery",
    "Left Anterior Cerebral Artery",
    "Right Anterior Cerebral Artery",
    "Left Posterior Communicating Artery",
    "Right Posterior Communicating Artery",
    "Basilar Tip",
    "Other Posterior Circulation",
    "Aneurysm Present",
]
set_seed(42)
ckpt_path ='/kaggle/input/resnet18-75epochs/best_model.pt'
device ='cuda'
num_classes = 14
USE_AMP=True
DEVICE ='cuda'
from torch import amp
@torch.no_grad()
def predict(series_path: str) -> pl.DataFrame:
    """Server calls this function. Assumes global `model` and `processor` are ready."""

    model = _load_model_for_fold(resnet18, ckpt_path, device, num_classes).float()
    import torch.nn as nn

    # Change conv1 from in_channels=1 → 2   <===this is added

    # Replace conv1 with in_channels=2
    '''
    model.conv1 = nn.Conv3d(
        in_channels=2,
        out_channels=64,
        kernel_size=(7, 7, 7),
        stride=(2, 2, 2),
        padding=(3, 3, 3),
        bias=False
    ).to(device).type(next(model.parameters()).dtype)  # <-- match dtype

    model.conv1 = nn.Conv3d(
    in_channels=1, out_channels=64,
    kernel_size=(7, 7, 7), stride=(2, 2, 2), padding=(3, 3, 3), bias=False
     ).to(device).type(next(model.parameters()).dtype)  # <-- match dtype
    '''
    #model = model.float()
    #print('\\\\\\\\\\\\\\\\\\\\')
    try:
        # CPU preprocessing
        #print(f'process_series_to_npz={process_series_to_npz(series_path)}')
        volume = process_series_to_npz(series_path)
        #volume = load_series_as_array(series_path)
          # (D,H,W) in [0,1]
        volume_tensor = torch.from_numpy(volume).unsqueeze(0).to(DEVICE)  # (1,1,D,H,W)
        # Suppose x is your volume: [D,H,W]
        #volume  = torch.tensor(volume , dtype=torch.float32)  # [D,H,W]

        # Add channel and batch dimensions
        #volume = volume_tensor.unsqueeze(0)#.unsqueeze(0)           # [1,1,D,H,W]

        # Move to device
        #volume  = volume.to(DEVICE)

        print(f'volume shape ={volume.shape}')
        # Forward pass
        with amp.autocast(device_type='cuda', enabled=USE_AMP):
              logits = model(volume_tensor)# (1,14)
        # added code
        if isinstance(logits, tuple):   # e.g., (cls, seg)
            logits = logits[0]  # take classification output
            ##############

            logits = torch.sigmoid(logits)
            probs = torch.sigmoid(logits).cpu().numpy().flatten().tolist()

        result_df = pl.DataFrame(data=[probs], schema=LABEL_COLS, orient='row')

        # Cleanup
        del volume_tensor
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
        gc.collect()

    except Exception as e:
        print(f"[Predict] Error: {e}")
        result_df = pl.DataFrame(data=[[0.5] * len(LABEL_COLS)], schema=LABEL_COLS, orient='row')

    # Remove shared temp if present
    import shutil
    shutil.rmtree('/kaggle/shared', ignore_errors=True)
    return result_df


## 6-Run THe Evaluation Server:


In [ ]:
import sys
try:
          import kaggle_evaluation.rsna_inference_server
          inference_server = kaggle_evaluation.rsna_inference_server.RSNAInferenceServer(predict)
except:
          sys.path.append('/kaggle/input/rsna-intracranial-aneurysm-detection/kaggle_evaluation')
          import kaggle_evaluation.rsna_inference_server
          inference_server = kaggle_evaluation.rsna_inference_server.RSNAInferenceServer(predict)

#sys.path.append('/kaggle/input/rsna-intracranial-aneurysm-detection/kaggle_evaluation')

import shutil, os

#shared_dir = "/kaggle/shared"
#if os.path.exists(shared_dir):
    #shutil.rmtree(shared_dir)   # delete it completely
#os.makedirs(shared_dir, exist_ok=True)
if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
    inference_server.serve()
else:
    sys.path.append('/kaggle/input/rsna-intracranial-aneurysm-detection/kaggle_evaluation')
    inference_server.run_local_gateway()
    try:
        sub = pl.read_parquet('/kaggle/working/submission.parquet')
        import pandas as pd

        #sub = pd.read_parquet("submission.parquet")
        #print(sub.dtypes)
        #print(sub.head())
        #print("Shape:", sub.shape)
        #sub = pl.read_parquet('/kaggle/working/submission.parquet')
        print(sub.head())
    except Exception as e:
        print(f"Submission parquet not found yet: {e}")